# Python Intro - Final Exercise


In [215]:
import pyodbc
import pandas as pd
import numpy as np

In [216]:
### In windows, Using a ODBC DNS (predefined connection name)
conn = pyodbc.connect("DSN=College;"
                      "Server=DESKTOP-E6QQ4LT\SQLEXPRESS;"
                      "Database=College;"
                      "Trusted_Connection=yes;")


In [217]:
### Get the students table
students = pd.read_sql("SELECT * FROM Students;",conn) 
students.head(0)

,Student_id,First_name,Last_name,Gender


In [218]:
departments = pd.read_sql("SELECT * FROM Departments;",conn)
departments.head(0)

,Department_id,Department_name


In [219]:
courses = pd.read_sql("SELECT * FROM Courses;",conn)
courses.head(0)

,Course_id,Course_name,Department_id,Teacher_id


In [220]:
classroom = pd.read_sql("SELECT * FROM Classroom;",conn)
classroom.head(0)

,Course_id,Student_id,Degree


In [221]:
teachers = pd.read_sql("SELECT * FROM Teachers;",conn)
teachers.head(0)

,Teacher_id,First_name,Last_name,Gender


# 1. Count the number of students on each department

In [246]:
#inner join classroom with courses
classroom_courses = pd.merge(classroom, 
                             courses[['Course_id', 'Course_name', 'Department_id']], 
                             on='Course_id')

#inner join classroom_courses with departments 
classroom_courses_departments = pd.merge(classroom_courses,
                                        departments[['Department_id', 'Department_name']],
                                        on='Department_id')

#counting the number of students in each department
classroom_courses_departments.groupby('Department_id').agg({'Student_id': pd.Series.nunique})

,Student_id
Department_id,
1,84
2,200
3,137
4,81


# 2. How many students have each course of the English department and the total number of students in the department?

In [263]:
# selecting only the English department
english_dep = classroom_courses_departments.copy()
english_dep=english_dep[classroom_courses_departments.Department_id==1]
#counting the number of student in each course

english = english_dep.groupby(['Department_id','Course_name']).agg({'Student_id': [pd.Series.nunique]})
english.reset_index(inplace=True)
english.columns = ['Department_id','Course_name','TotalStudentInDepartment']
english.head()


,Department_id,Course_name,TotalStudentInDepartment
0,1,Advanced English,24
1,1,English Begginers,32
2,1,Proffesional English,38


In [264]:

#counting total student in the english department
TotalStudent = english_dep.groupby('Department_id').agg({'Student_id': pd.Series.nunique})
TotalStudent.columns = ['TotalStudent']
TotalStudent

,TotalStudent
Department_id,
1,84


In [266]:

english.reset_index(inplace=True)
TotalStudent.reset_index(inplace=True)
TotalStudent_english = pd.merge(english,TotalStudent, on='Department_id')
TotalStudent_english.head()

,index,Department_id,Course_name,TotalStudentInDepartment,TotalStudent
0,0,1,Advanced English,24,84
1,1,1,English Begginers,32,84
2,2,1,Proffesional English,38,84


# 3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

In [226]:
#selecting the Science department and agg. the student in each course
science_class =  classroom_courses_departments[classroom_courses_departments.Department_id.isin([2])].\
groupby(['Department_id','Course_id']).\
agg({'Student_id':pd.Series.nunique }) 
science_class.columns=['TotalInClass']


#class size by categ
science_class_cat = science_class.copy()
science_class_cat['ClassSize'] = ''
science_class_cat.loc[(science_class_cat['TotalInClass']< 22),'ClassSize']='Small'
science_class_cat.loc[(science_class_cat['TotalInClass']> 21),'ClassSize']='Big'


#counting the number of each class size
science_class_cat.groupby('ClassSize').agg({'ClassSize':['count']})

,ClassSize
,count
ClassSize,
Big,10
Small,6


# 4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

In [227]:
#counting the number of each gender 
students.groupby('Gender').agg({'Gender':['count']})

,Gender
,count
Gender,
F,165
M,115


# 5. For which courses the percentage of male/female students is over 70%?

In [228]:
#join classroom_courses with students
courses_classroom_students = pd.merge(classroom_courses,
                            students[['Gender','Student_id']],
                            on='Student_id')

#agg of student gender per course
corse_gender = courses_classroom_students.copy()
corse_gender = corse_gender.groupby(['Course_name','Gender']).\
agg({'Student_id':pd.Series.nunique})
corse_gender.columns=['TotalGender']

#creating a new column containing the total per course
corse_gender['TotalInClass'] = corse_gender.groupby('Course_name').transform('sum')

#creating a new column containing the percentage
corse_gender['Ratio'] = corse_gender.TotalGender / corse_gender.TotalInClass

#selecting the >70% courses
corse_gender = corse_gender[corse_gender.Ratio>0.7]
corse_gender.head(10)

#presenting the %
corse_gender['Percent'] = corse_gender['Ratio']*100
corse_gender.head(10)

,,TotalGender,TotalInClass,Ratio,Percent
Course_name,Gender,,,,
Genetics,F,26,37,0.702703,70.270270
Sculpture,F,17,23,0.739130,73.913043
Tenis,F,13,18,0.722222,72.222222


## Q6. For each department, how many students passed with a grades over 80?

In [229]:
#merge of class with courses
classroom_courses_degree = pd.merge(classroom, 
                             courses[['Course_id', 'Course_name', 'Department_id']], 
                             on='Course_id')

#selecting the student who have grade over 80
degree_80 = classroom_courses_degree[classroom_courses_degree.Degree>80]

#counting the student
degree_80 = degree_80.groupby('Department_id').agg({'Student_id':pd.Series.nunique})
degree_80.head()


,Student_id
Department_id,
1,40
2,122
3,61
4,39


## Q7. For each department, how many students passed with a grades under 60?

In [230]:
#selecting the student who have grade under 60
degree_60 = classroom_courses_degree[classroom_courses_degree.Degree<60]

#counting the student
degree_60 = degree_60.groupby('Department_id').agg({'Student_id':pd.Series.nunique})


degree_60.head()

,Student_id
Department_id,
1,2
2,7
3,3
4,1


# 8. Rate the teachers by their average student's grades (in descending order).

In [231]:
#shorter way of merging 3 tables
teacher_degree = pd.merge(pd.merge(teachers,courses, on='Teacher_id'),classroom, on='Course_id')
        
#calculting the avarage of each teacher
teacher_deg = teacher_degree.groupby(['Teacher_id','First_name','Last_name']).\
agg({'Degree':'mean'})

teacher_deg.reset_index(inplace=True)
teacher_deg.columns = ['Teacher_id','First_name','Last_name','Degree_mean']

#sort by degree(descending)
teacher_deg.sort_values(ascending=False,by='Degree_mean')


,Teacher_id,First_name,Last_name,Degree_mean
19,22,Darick,Hess,81.432703
2,3,Jordyn,Baron,80.729494
3,5,Heather,Metcalf,80.440242
17,20,Zachary,Healy,79.365867
0,1,Kaylea,Monzingo,79.365537
15,18,Daniel,Bland,78.908281
21,24,Keegan,Price,78.814631
12,15,Conor,Mckittrick,78.699492
20,23,Maxwell,Romkee,78.673509
16,19,Terran,Beaty,78.495479


## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

In [232]:
#merging the tables
df1_cour_dep = pd.merge(pd.merge(pd.merge(courses,
                       departments,
                       on='Department_id'),teachers,on='Teacher_id'),classroom,on='Course_id')

df1_cour_dep['Teachers_name'] = df1_cour_dep.First_name +' '+ df1_cour_dep.Last_name
#grouping by courses and counting the number of students in each course
df2_cour_dep = df1_cour_dep.copy()
df2_cour_dep = df2_cour_dep.groupby(['Course_name','Department_name','Teachers_name']).agg({'Student_id':'count'}) 
df2_cour_dep.reset_index(inplace=True)
df2_cour_dep.columns = ['Course_name','Department_name','Teachers_name','Sum_student_in_class']
df2_cour_dep.head()


,Course_name,Department_name,Teachers_name,Sum_student_in_class
0,Advanced English,English,Conor Mckittrick,24
1,Algebra,Science,Alexander Watts,12
2,Biology,Science,Jordyn Baron,36
3,Chemistry,Science,Jeffrey Batzel,24
4,Computer Science,Science,Kaylea Monzingo,38


# 10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).

In [233]:

df_student_course = pd.merge(pd.merge(pd.merge(students,\
                                                classroom,how='inner',on='Student_id'),\
                                       courses,how='inner',on='Course_id'),\
                              departments,how='inner',on='Department_id')


df1_student_course = df_student_course.copy()
df1_student_course = df1_student_course.groupby('Student_id').agg({'Degree':['mean']})
df1_student_course.reset_index(inplace=True)
df1_student_course.columns = ['Student_id','Total_degree']
df1_student_course.head()

df2_student_course = df_student_course.copy()
df2_student_course = df2_student_course.groupby(['Student_id','Department_name']).agg({'Degree':['mean']})
df2_student_course.reset_index(inplace=True)
df2_student_course.columns = ['Student_id','Department_name','Department_Avr']
df2_student_course.head()


df3_student_course = df_student_course.copy()
df3_student_course = df3_student_course.groupby(['Student_id','First_name','Last_name']).agg({'Course_name':['count']})
df3_student_course.reset_index(inplace=True)
df3_student_course.columns = ['Student_id','First_name','Last_name','Sum_courses']
df3_student_course.head()



,Student_id,First_name,Last_name,Sum_courses
0,1,Alexandria,Heller,4
1,2,Cassie,Liska,5
2,3,Hayley,Frank,3
3,4,Maxwell,Kotch,3
4,5,Dylan,Boehm,3


In [275]:
df4_student_course = pd.merge(df1_student_course,df2_student_course,on='Student_id')
df4_student_course.head()
df4_student_course.reset_index(inplace=True)

df6_student_course = pd.merge(df4_student_course,df3_student_course)
#df6_student_course.head()


df6_student_course.pivot_table(index=['First_name','Last_name'], columns=['Department_name'], values=['Total_degree','Sum_courses','Department_Avr'])

Department_Avr                                   \
Department_name                    Arts    English    Science      Sport   
First_name  Last_name                                                      
 Aaron      Dejoy             73.889151        NaN  63.563020  83.765738   
 Abbey      Singley                 NaN  80.199500        NaN  65.727646   
 Adam       Stubblefield      73.890860        NaN  81.892666        NaN   
 Adrianna   Bean                    NaN  81.063373  74.996719        NaN   
 Alec       Miller                  NaN  73.691636        NaN  84.158496   
 Alexandrea Hannafious              NaN  89.626782        NaN  85.638019   
 Alexandria Heller            86.677535        NaN  79.557508        NaN   
 Alexis     Cruse             84.401408  66.022745  71.432893        NaN   
 Alia       Jones                   NaN        NaN  86.512810        NaN   
 Alicia     Schlichting             NaN  78.568032  81.337292        NaN   
 Alissa     Marsh                   NaN        NaN  77.412190        NaN   
 Allison    Otey              58.413779        NaN        NaN        NaN   
 Alyssa     Fabian            72.765028  74.388933  83.709314        NaN   
            Grounds           80.951105  73.225274  84.726407  66.312639   
            Thornton          74.124803        NaN        NaN        NaN   
            Zarazua           74.267760  66.909443  82.637933  85.782442   
 Alyx       Hufford           72.070930        NaN  84.341710  81.147429   
 Amanda     Thomas                  NaN  54.835800        NaN        NaN   
 Andrew     Holiday                 NaN  74.680466  73.482645        NaN   
            Porter            75.746633  77.508237        NaN        NaN   
 Angela     Palermo           83.056965        NaN        NaN        NaN   
 Anina      Meneses           80.183240        NaN  82.269726  78.507968   
 Anna       Cain                    NaN        NaN  80.389515        NaN   
 Ashleigh   Conder            69.034424  81.015023  81.606452        NaN   
 Ashley     Beziou            82.838731        NaN  67.076918        NaN   
            Cannon                  NaN        NaN  79.014799  82.912853   
            Walkey                  NaN        NaN  71.048747        NaN   
 Ashton     Dixon                   NaN        NaN  86.349044        NaN   
 Asia       Roberts           72.905192        NaN        NaN        NaN   
 Austin     Jones             78.840946        NaN        NaN  81.764484   
...                                 ...        ...        ...        ...   
 Savanna    Crochet           68.709468        NaN  79.211718        NaN   
 Sean       Hasenack          96.216926  89.073972  79.304200        NaN   
 Shaun      Krasovic                NaN        NaN  83.934178        NaN   
 Shawn      Foster            96.208535        NaN  75.593857        NaN   
 Shawna     East              84.298766        NaN  77.827647        NaN   
 Shelby     Lucio             89.668959        NaN  86.938528        NaN   
            Robinson                NaN        NaN  81.741539        NaN   
 Shepherd   Gillham           78.806038  73.163042  68.051481        NaN   
 Sierra     Van Vorst               NaN        NaN  82.592693        NaN   
 Skylar     Blake                   NaN        NaN  97.903656        NaN   
 Skyler     Norris            86.901030        NaN        NaN        NaN   
 Sophiya    Aponte            87.463154  83.145779  61.489085        NaN   
 Sydney     Blas                    NaN        NaN  72.578017  76.749775   
 Taiahbeth  Gerard            78.695946        NaN        NaN        NaN   
 Tamara     Webber            82.399676        NaN  58.176686        NaN   
 Tamera     Engel                   NaN        NaN  82.740547  80.114350   
 Tarah      Gunnett                 NaN        NaN  86.795013        NaN   
 Trenton    Goter             88.339007        NaN  75.933173        NaN   
 Trevor     Cook              75.484903        NaN  80.331178  86.174758 

In [ ]:
היי תומאס ניסיתי להפוך את הטבלה מעמודות לשורות אבל לא הצלחתי. אשמח לעזרה